In [1]:
import os

cwd = os.getcwd()
directory = os.path.join(cwd + "/clean_data/")
file_names = []
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith(".csv"):
            file_names.append(str(file))
#print(file_names)

def mid_3chars(x):
    return(x[2:5])

file_names_sorted = [filename for filename in sorted(file_names, key = mid_3chars)]
print(file_names_sorted)

['00011.csv', '00012.csv', '00013.csv', '00014.csv', '00015.csv', '00016.csv', '00017.csv', '00018.csv', '00020.csv', '00021.csv', '00022.csv', '00023.csv', '00024.csv', '00025.csv', '00026.csv', '00027.csv', '00028.csv', '00029.csv', '00030.csv', '00031.csv', '00032.csv', '00033.csv', '00034.csv', '00035.csv', '00036.csv', '00037.csv', '00038.csv', '00039.csv', '00040.csv', '00041.csv', '00042.csv', '00043.csv', '00044.csv', '00045.csv', '00046.csv', '00047.csv', '00048.csv', '00049.csv', '00050.csv', '00051.csv', '00052.csv', '00053.csv', '00054.csv', '00055.csv', '00056.csv', '00057.csv', '00058.csv', '00059.csv', '00060.csv', '00061.csv', '00062.csv', '00063.csv', '00064.csv', '00065.csv', '00066.csv', '00067.csv', '00068.csv', '00069.csv', '00070.csv', '00071.csv', '00072.csv', '00073.csv', '00074.csv', '00075.csv', '00076.csv', '00077.csv', '00078.csv', '00079.csv', '00080.csv', '00081.csv', '00082.csv', '00083.csv', '00084.csv', '00085.csv', '00086.csv', '00087.csv', '00088.csv'

In [3]:
# split the user id - training section and testing section
import pandas as pd

split_date = '2017-04-28'
split_previous_user_id = []
split_after_user_id = []
for file_name_sorted in file_names_sorted:
    df = pd.read_csv(directory + file_name_sorted)
    split_previous_user_id = split_previous_user_id + df[(df.date <= split_date)].uid.tolist()
    split_previous_user_id = list(set(split_previous_user_id))
    split_after_user_id = split_after_user_id + df[(df.date > split_date)].uid.tolist()
    split_after_user_id = list(set(split_after_user_id))

print('the number of unique user id in training set:' + str(len(split_previous_user_id)))
print('the number of unique user id in training set:' + str(len(split_after_user_id)))

the number of unique user id in training set:529700
the number of unique user id in training set:214323


In [4]:
# shown before the threshold but not shown after the threshold  
print('the number of unique user id only shown in training set:' + str(len(list(set(split_previous_user_id) - set(split_after_user_id)))))

the number of unique user id only shown in training set:329901


In [5]:
# shown after the threshold but not shown before the threshold  
print('the number of unique user id only shown in testing set:' + str(len(list(set(split_after_user_id) - set(split_previous_user_id)))))

the number of unique user id only shown in testing set:14524


In [6]:
# remove the uid shown after the threshold but not shown before the threshold 
split_after_remove_id = list(set(split_after_user_id) - set(split_previous_user_id))
split_after_user_id_remove_new_user = list(set(split_after_user_id) - set(split_after_remove_id))
print('the number of unique user id shown in both training set and testing set:' + str(len(split_after_user_id_remove_new_user)))

the number of unique user id shown in both training set and testing set:199799


In [7]:
# ratio of target - 1 (churn) versus 0 (not churn)
# remove user id we don't care about
terget_0_num = len(split_after_user_id_remove_new_user)
terget_1_num = len(split_previous_user_id) - len(split_after_user_id_remove_new_user)
print("The percentage of target label with not churn and churn = %.4f and %.4f"%(terget_0_num/(terget_0_num + terget_1_num), terget_1_num/(terget_0_num + terget_1_num)))

The percentage of target label with not churn and churn = 0.3772 and 0.6228


Feature Engineering 

Target Label: 1/0(churn / not churn) Down sampling to make it as a balanced problem

Features:
1. 1 day frequency on play log(# of record in playing log for each user id since 1 day before snapdate)
2. 3 day frequency on play log(# of record in playing log for each user id since 3 day before snapdate) 
3. 7 day frequency on play log(# of record in playing log for each user id since 7 day before snapdate) 
4. 14 day frequency on play log(# of record in playing log for each user id since 14 day before snapdate)
5. 30 day frequency on play log(# of record in playing log for each user id since 30 day before snapdate)
6. Recency on play log
7. completeness of listening activity (mean(total_play_time/total_song_length))

In [15]:
import datetime
snapdate = datetime.date(2017, 4 ,28)
N = [1,3,7,14,30]
date_N_days_ago = snapdate - datetime.timedelta(days=N[0])
print(date_N_days_ago)

2017-04-27


In [21]:
# combine the each subfile into one
frame = pd.DataFrame()
list_ = []
for file_name_sorted in file_names_sorted:
    df = pd.read_csv(directory + file_name_sorted, header=0, encoding='utf-8', index_col=False)
    print(str(int(file_names_sorted.index(file_name_sorted)) + 1) + ' files are parsed!')
    list_.append(df)
frame = pd.concat(list_)
whole_file = 'whole_clean_data.csv'
frame.to_csv(directory + whole_file, encoding='utf-8', index=False, header = ['date', 'uid', 'play_time', 'song_length', 'song_id'])

1 files are parsed!
2 files are parsed!
3 files are parsed!
4 files are parsed!
5 files are parsed!
6 files are parsed!
7 files are parsed!
8 files are parsed!
9 files are parsed!
10 files are parsed!
11 files are parsed!
12 files are parsed!
13 files are parsed!
14 files are parsed!
15 files are parsed!
16 files are parsed!
17 files are parsed!
18 files are parsed!
19 files are parsed!
20 files are parsed!
21 files are parsed!
22 files are parsed!
23 files are parsed!
24 files are parsed!
25 files are parsed!
26 files are parsed!
27 files are parsed!
28 files are parsed!
29 files are parsed!
30 files are parsed!
31 files are parsed!
32 files are parsed!
33 files are parsed!
34 files are parsed!
35 files are parsed!
36 files are parsed!
37 files are parsed!
38 files are parsed!
39 files are parsed!
40 files are parsed!
41 files are parsed!
42 files are parsed!
43 files are parsed!
44 files are parsed!
45 files are parsed!
46 files are parsed!
47 files are parsed!
48 files are parsed!
4